In [ ]:
!pip install -q bezpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import bezpy as bz
import geopandas as gpd
from geopy.distance import great_circle

## Horton grid
The data for the substations, transmission lines, and transformers are derived from this [work](https://ieeexplore.ieee.org/document/6298994). Setting NERC to true uses the grid data from the NERC documentation on solving GIC using the Nodal Admittance Matrix.


In [23]:
# Large val to replace infs
large_vals = 1e10
NERC = False # Use the test case in NERC docs or D.Boteler / Horton 2013 Test Case

# Transmission line data
lines = [
    {"name": "1", "from_bus": 2, "to_bus": 3, "R": 3.512, "V": 345},
    {"name": "2", "from_bus": 2, "to_bus": 17, "R": 3.525, "V": 345},
    {"name": "3", "from_bus": 15, "to_bus": 4, "R": 1.986, "V": 500},
    {"name": "4", "from_bus": 17, "to_bus": 16, "R": 4.665, "V": 345},
    {"name": "5", "from_bus": 4, "to_bus": 5, "R": 2.345, "V": 500},
    {"name": "6", "from_bus": 4, "to_bus": 5, "R": 2.345, "V": 500},
    {"name": "7", "from_bus": 5, "to_bus": 6, "R": 2.975, "V": 500},
    {"name": "8", "from_bus": 5, "to_bus": 11, "R": 3.509 + large_vals, "V": 500},
    {"name": "9", "from_bus": 6, "to_bus": 11, "R": 1.444, "V": 500},
    {"name": "10", "from_bus": 4, "to_bus": 6, "R": 4.666, "V": 500},
    {"name": "11", "from_bus": 15, "to_bus": 6, "R": 2.924, "V": 500},
    {"name": "12", "from_bus": 15, "to_bus": 6, "R": 2.924, "V": 500},
    {"name": "13", "from_bus": 11, "to_bus": 12, "R": 2.324, "V": 500},
    {"name": "14", "from_bus": 16, "to_bus": 20, "R": 4.049, "V": 345},
    {"name": "15", "from_bus": 17, "to_bus": 20, "R": 6.940, "V": 345}
]

# Transformer data
transformers = [
    {"name": "T1", "type": "GSU w/ GIC BD", "W1": 0.1, "bus1": 2, "W2": 0.1, "bus2": 1},
    {"name": "T2", "type": "GY-GY-D", "W1": 0.2, "bus1": 4, "W2": 0.1, "bus2": 3},
    {"name": "T3", "type": "GSU", "W1": 0.1, "bus1": 17, "W2": large_vals, "bus2": 18},
    {"name": "T4", "type": "GSU", "W1": 0.1, "bus1": 17, "W2": large_vals, "bus2": 19},
    {"name": "T5", "type": "Auto", "W1": 0.04, "bus1": 15, "W2": 0.06, "bus2": 16},
    {"name": "T6", "type": "GSU", "W1": 0.15, "bus1": 6, "W2": large_vals, "bus2": 7},
    {"name": "T7", "type": "GSU", "W1": 0.15, "bus1": 6, "W2": large_vals, "bus2": 8},
    {"name": "T8", "type": "GY-GY", "W1": 0.04, "bus1": 5, "W2": 0.06, "bus2": 20},
    {"name": "T9", "type": "GY-GY", "W1": 0.04, "bus1": 5, "W2": 0.06, "bus2": 20},
    {"name": "T10", "type": "GSU", "W1": 0.1, "bus1": 12, "W2": large_vals, "bus2": 13},
    {"name": "T11", "type": "GSU", "W1": 0.1, "bus1": 12, "W2": large_vals, "bus2": 14},
    {"name": "T12", "type": "Auto", "W1": 0.04, "bus1": 4, "W2": 0.06, "bus2": 3},
    {"name": "T13", "type": "GY-GY-D", "W1": 0.2, "bus1": 4, "W2": 0.1, "bus2": 3},
    {"name": "T14", "type": "Auto", "W1": 0.04, "bus1": 4, "W2": 0.06, "bus2": 3},
    {"name": "T15", "type": "Auto", "W1": 0.04, "bus1": 15, "W2": 0.06, "bus2": 16},
    # {"name": "T16", "type": "Tee", "W1":0.01, "bus1": 6, "W2":0.01, "bus2": 11}
]

# Substation data
substations = {
    1: {'name': 'Substation 1', 'buses': [1, 2], 'transformers': ['T1'], 'grounding_resistance': 0.2, 'latitude': 33.6135, 'longitude': -87.3737},
    2: {'name': 'Substation 2', 'buses': [17, 18, 19], 'transformers': ['T3', 'T4'], 'grounding_resistance': 0.2, 'latitude': 34.3104, 'longitude': -86.3658},
    3: {'name': 'Substation 3', 'buses': [15, 16], 'transformers': ['T15', 'T5'], 'grounding_resistance': 0.2, 'latitude': 33.9551, 'longitude': -84.6794},
    4: {'name': 'Substation 4', 'buses': [3, 4], 'transformers': ['T2', 'T12', 'T13', 'T14'], 'grounding_resistance': 1, 'latitude': 33.5479, 'longitude': -86.0746},
    5: {'name': 'Substation 5', 'buses': [5, 20], 'transformers': ['T8', 'T9'], 'grounding_resistance': 0.1, 'latitude': 32.7051, 'longitude': -84.6634},
    6: {'name': 'Substation 6', 'buses': [6, 7, 8], 'transformers': ['T6', 'T7'], 'grounding_resistance': 0.1, 'latitude': 33.3773, 'longitude': -82.6188},
    7: {'name': 'Substation 7', 'buses': [11], 'transformers': ['T16'], 'grounding_resistance': 0.1, 'latitude': 34.2522, 'longitude': -82.8363},
    8: {'name': 'Substation 8', 'buses': [12, 13, 14], 'transformers': ['T10', 'T11'], 'grounding_resistance': 0.1, 'latitude': 34.1956, 'longitude': -81.098}
}

if NERC:
  # NERC lines
  lines = [
      {"name": "1", "from_bus": 2, "to_bus": 3, "R": 3.525, "V": 345},
      {"name": "2", "from_bus": 4, "to_bus": 5, "R": 4.665, "V": 345},
  ]
  # NERC transformers
  transformers = [
      {"name": "T1", "type": "GSU", "W1": 0.5, "bus1": 2, "W2": large_vals, "bus2": 1},
      {"name": "T2", "type": "Auto", "W1": 0.2, "bus1": 4, "W2": 0.2, "bus2": 3},
      # {"name": "T3", "type": "Auto", "W1": 0.2, "bus1": 4, "W2": 0.2, "bus2": 3},
      {"name": "T4", "type": "GSU", "W1": 0.5, "bus1": 5, "W2": large_vals, "bus2": 6},
  ]
  # NERC substations
  substations = {
      1: {'name': 'Substation 1', 'buses': [1,2], 'transformers': ['T1'], 'grounding_resistance': 0.2, 'latitude': 33.6135, 'longitude': -87.3737},
      2: {'name': 'Substation 2', 'buses': [3,4], 'transformers': ['T2'], 'grounding_resistance': 0.2, 'latitude': 34.310437, 'longitude': -86.3658},
      3: {'name': 'Substation 3', 'buses': [5,6], 'transformers': ['T3'], 'grounding_resistance': 0.2, 'latitude': 33.955058, 'longitude': -84.679354},
  }

df_lines = pd.DataFrame(lines)
df_transformers = pd.DataFrame(transformers)
substations_df = pd.DataFrame(substations).T
df_lines

,name,from_bus,to_bus,R,V
0,1,2,3,3.512000e+00,345
1,2,2,17,3.525000e+00,345
2,3,15,4,1.986000e+00,500
3,4,17,16,4.665000e+00,345
4,5,4,5,2.345000e+00,500
5,6,4,5,2.345000e+00,500
6,7,5,6,2.975000e+00,500
7,8,5,11,1.000000e+10,500
8,9,6,11,1.444000e+00,500
9,10,4,6,4.666000e+00,500


The DataFrame indicates the transmission lines and the connecting buses. $R$ represents the resistances of the lines, and $V$ denotes the voltage levels of the transmission lines.


## Distances between the substations
The distances between the substations are computed using the haversine distance formula, which takes the coordinates of the substations as input. The formula calculates the eastern (x) and northern (y) displacement vectors of the transmission lines. These values are multiplied by the geoelectric field, $E \cdot dl$, to compute the electrostatic potentials (electromotive forces) of the lines.


In [26]:
def haversine_distance_and_components(lat1, lon1, lat2, lon2):
    # Earth's radius in km
    R = 6371

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c

    # Calculate x and y component%
    # Increase length by 35
    dx = R * np.cos(lat1) * (lon2 - lon1)
    dy = R * (lat2 - lat1)

    # assert np.sqrt(dx**2 + dy**2) == distance, "Distance calculation is incorrect"

    # Calculate E-field components (assuming 1 V/km uniform field if not specified)
    E_x = dx
    E_y = dy

    return distance, E_x, E_y

# Point 1 is substation 1
point1 = substations_df.iloc[0]
point2 = substations_df.iloc[1]

distance, dx, dy = haversine_distance_and_components(
        point1['latitude'], point1['longitude'],
        point2['latitude'], point2['longitude'])

## Eastern and Northward Components of the Geoelectric Fields
The Horton grid applies a uniform geoelectric field of 1 V/km, which can be adjusted to test different values.

In [27]:
def find_substation_name(bus, sub_ref):
    for sub_name, buses in sub_ref.items():
        if bus in buses:
            return sub_name
    return None

# Create a dictionary for quick substation lookup
sub_ref = dict(zip(substations_df.name, substations_df.buses))

# Add origin and to substations
df_lines['origin_sub'] = df_lines.from_bus.apply(lambda x: find_substation_name(x, sub_ref))
df_lines['to_sub'] = df_lines.to_bus.apply(lambda x: find_substation_name(x, sub_ref))

# Create a dictionary for quick latitude and longitude lookup
sub_coords = substations_df.set_index('name')[['latitude', 'longitude']]

# Calculate Ex, Ey, and distance for each line
df_lines[['distance', 'Ex', 'Ey']] = df_lines.apply(
    lambda row: haversine_distance_and_components(
        sub_coords.loc[row.origin_sub, 'latitude'],
        sub_coords.loc[row.origin_sub, 'longitude'],
        sub_coords.loc[row.to_sub, 'latitude'],
        sub_coords.loc[row.to_sub, 'longitude']
    ),
    axis=1, result_type='expand'
)

# Display the updated df_lines
df_lines

,name,from_bus,to_bus,R,V,origin_sub,to_sub,distance,Ex,Ey
0,1,2,3,3.512000e+00,345,Substation 1,Substation 4,120.565226,120.299408,-7.294387
1,2,2,17,3.525000e+00,345,Substation 1,Substation 2,121.017407,93.333672,77.491744
2,3,15,4,1.986000e+00,500,Substation 3,Substation 4,136.705787,-128.684139,-45.278574
3,4,17,16,4.665000e+00,345,Substation 2,Substation 3,160.163799,154.890045,-39.507557
4,5,4,5,2.345000e+00,500,Substation 4,Substation 5,161.403581,130.779477,-93.715084
5,6,4,5,2.345000e+00,500,Substation 4,Substation 5,161.403581,130.779477,-93.715084
6,7,5,6,2.975000e+00,500,Substation 5,Substation 6,204.710132,191.305825,74.745230
7,8,5,11,1.000000e+10,500,Substation 5,Substation 7,241.464899,170.955137,172.029671
8,9,6,11,1.444000e+00,500,Substation 6,Substation 7,99.337833,-20.195982,97.284441
9,10,4,6,4.666000e+00,500,Substation 4,Substation 6,321.119288,320.257737,-18.969854


The DataFrame comprises the distances of the transmission lines ($distance$) and the eastward ($E_x$) and northward ($E_y$) components of the geoelectric fields, assuming a value of 1 V/km.


## Transformers Data Structures

The transformer termination points are the buses which serve as the source and destination points of transmission lines connecting to adjacent substations. Each evaluated transformer has primary (serial for autotransformers) and secondary winding (common for autotransformers) resistances. For GSU transformers, we assign infinite resistances to the secondary windings.

For each substation, we add neutral points (ground nodes). Following the Lehtinen-Pirjola Modified methodology, we introduce virtual grounded nodes to ungrounded transformers of delta types and GSUs. We assign infinite resistances to these transformers (zero admittances/conductances).

The general impedance equation is:

$$Z=R+jX$$

However, we neglect the inductances and capacitances of the transmission lines, therefore:

$$Z=R$$

And consequently, the conductances are equivalent to admittances:

$$G=Y=1/R$$

In [31]:
sub_look_up = {}
index = 0
for i, row in substations_df.iterrows():
  buses = row["buses"]
  for bus in sorted(buses):
    sub_look_up[bus] = index
    index+=1

for i, row in substations_df.iterrows():
  if row["name"] !=  'Substation 7':
    sub_look_up[row["name"]] = index
    index += 1

# Neutral points in a trafor
df_transformers["sub"] = df_transformers.bus1.apply(lambda x: find_substation_name(x, sub_ref))
df_transformers["neutral_point"] = df_transformers["sub"].apply(lambda x: sub_look_up.get(x, None))
df_transformers

,name,type,W1,bus1,W2,bus2,sub,neutral_point
0,T1,GSU w/ GIC BD,0.10,2,1.000000e-01,1,Substation 1,18
1,T2,GY-GY-D,0.20,4,1.000000e-01,3,Substation 4,21
2,T3,GSU,0.10,17,1.000000e+10,18,Substation 2,19
3,T4,GSU,0.10,17,1.000000e+10,19,Substation 2,19
4,T5,Auto,0.04,15,6.000000e-02,16,Substation 3,20
5,T6,GSU,0.15,6,1.000000e+10,7,Substation 6,23
6,T7,GSU,0.15,6,1.000000e+10,8,Substation 6,23
7,T8,GY-GY,0.04,5,6.000000e-02,20,Substation 5,22
8,T9,GY-GY,0.04,5,6.000000e-02,20,Substation 5,22
9,T10,GSU,0.10,12,1.000000e+10,13,Substation 8,24


We introduce virtual neutral points to the ungrounded substations comprising GSU or purely delta-delta transformers.

## Admittance Matrix ($Y^n$)
We build the network admittance matrix $Y^n$ excluding the grounding node impedances

In [36]:
# Build Y^n
# Number of unique nodes (buses + neutral points)
n_nodes = len(sub_look_up)

# Initialize the admittance matrix Y
Y = np.zeros((n_nodes, n_nodes))

phases = 1

def add_admittance(Y, from_bus, to_bus, admittance):
    i, j = from_bus, to_bus
    Y[i, i] += admittance
    if i != j:
        Y[j, j] += admittance
        Y[i, j] -= admittance
        Y[j, i] -= admittance

def add_admittance_auto(Y, from_bus, to_bus, neutral_bus, Y_series, Y_common):
    i, j, k = to_bus, from_bus, neutral_bus
    add_admittance(Y, from_bus, neutral_bus, Y_common)
    add_admittance(Y, from_bus, to_bus, Y_series)

    Y[i, i] += Y_common
    Y[i, i] += Y_series
    Y[j, j] += Y_series
    Y[i, j] -= Y_series
    Y[j, i] -= Y_series

# Process transformers and build admittance matrix
for bus, bus_idx in sub_look_up.items():
    sub = find_substation_name(bus, sub_ref)

    # Filter transformers for current bus
    trafos = df_transformers[(df_transformers["bus1"] == bus)]

    if len(trafos) == 0 or sub == "Substation 7":
        continue

    # Process each transformer
    for _, trafo in trafos.iterrows():
        # Extract parameters
        bus1 = trafo["bus1"]
        bus2 = trafo["bus2"]
        neutral_point = trafo["sub"]  # Neutral point node (for auto-transformers)
        W1 = trafo["W1"]  # Impedance for Winding 1 (Primary, Series)
        W2 = trafo["W2"]  # Impedance for Winding 2 (Secondary, if available)

        trafo_type = trafo["type"]
        bus1_idx = sub_look_up[bus1]
        neutral_idx = sub_look_up[neutral_point] if neutral_point in sub_look_up else None
        bus2_idx = sub_look_up[bus2]

        if trafo_type == "GSU":
            Y_w1 = 1 / W1  # Primary winding admittance
            add_admittance(Y, bus1_idx, neutral_idx, Y_w1)

        elif trafo_type == "Tee":
            # Y_pri = phases / (row['W1'])
            # Y_sec = phases / (row['W2'])
            # add_admittance(Y, bus_n, np_bus, Y_pri)
            # add_admittance(Y, bus_k, np_bus, Y_sec)
            continue

        elif trafo_type == "GSU w/ GIC BD":
            Y_w1 = 1 / W1  # Primary winding admittance
            add_admittance(Y, bus1_idx, neutral_idx, Y_w1)

        elif trafo_type == "Auto":
            Y_series = 1 / W1
            Y_common = 1 / W2
            add_admittance(Y, bus2_idx, bus1_idx, Y_series)
            add_admittance(Y, bus2_idx, neutral_idx, Y_common)

        elif trafo_type in ["GY-GY-D", "GY-GY"]:
            Y_primary = 1 / W1
            Y_secondary = 1 / W2
            add_admittance(Y, bus1_idx, neutral_idx, Y_primary)
            add_admittance(Y, bus2_idx, neutral_idx, Y_secondary)

# Add transmission line admittances
for i, line in df_lines.iterrows():
    Y_line = phases / line['R']
    bus_n = sub_look_up[line['from_bus']]
    bus_k = sub_look_up[line['to_bus']]
    add_admittance(Y, bus_n, bus_k, Y_line)

# Find indices of rows and columns where all elements are zero
non_zero_rows_cols = np.where(~np.all(Y == 0, axis=1))[0]  # Rows/cols with non-zero entries

# Create a reduced Y matrix by keeping only the non-zero rows/columns
Y_reduced = Y[non_zero_rows_cols[:, np.newaxis], non_zero_rows_cols]

y_cols = [i + 1 for i in range(Y_reduced.shape[0])]

Y_df = pd.DataFrame(np.round(Y, 2))
# Y_df.index = y_cols

In [34]:
# Find indices of rows/columns where all elements are zero
zero_row_indices = np.where(np.all(Y == 0, axis=1))[0]  # Zero rows
zero_col_indices = np.where(np.all(Y == 0, axis=0))[0]  # Zero columns

# Get the non-zero row/col indices
non_zero_indices = np.setdiff1d(np.arange(Y.shape[0]), zero_row_indices)

# Reduced Y matrix
Y_n = Y[np.ix_(non_zero_indices, non_zero_indices)]
pd.DataFrame(Y_n)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,10.568426,-0.283688,0.000000,0.000000,-0.284738,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,-10.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,-0.283688,20.642142,0.000000,-0.214362,0.000000,0.000000,0.000000e+00,-0.144092,0.000000,0.000000e+00,0.000000,0.0,-20.0,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000000,0.000000,51.187519,-50.000000,0.000000,-0.503525,0.000000e+00,0.000000,-0.683995,0.000000e+00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,-0.214362,-50.000000,83.794670,0.000000,0.000000,0.000000e+00,-0.246975,0.000000,0.000000e+00,0.000000,0.0,0.0,-33.333333,0.000000,0.000000,0.000000,0.0
4,-0.284738,0.000000,0.000000,0.000000,163.618071,-50.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,-113.333333,0.000000,0.000000,0.0
5,0.000000,0.000000,-0.503525,0.000000,-50.000000,91.570719,-8.528785e-01,0.000000,-0.214316,0.000000e+00,0.000000,0.0,0.0,0.000000,-40.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,-0.852878,2.011890e+02,0.000000,-0.336134,-1.000000e-10,0.000000,0.0,0.0,0.000000,0.000000,-200.000000,0.000000,0.0
7,0.000000,-0.144092,0.000000,-0.246975,0.000000,0.000000,0.000000e+00,133.724400,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.000000,-133.333333,0.000000,0.0
8,0.000000,0.000000,-0.683995,0.000000,0.000000,-0.214316,-3.361345e-01,0.000000,15.260299,-6.925208e-01,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-13.333333,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000e-10,0.000000,-0.692521,1.122813e+00,-0.430293,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


**$Y^n$ from LPm-paper**


In [ ]:
# Build earthing impedance Y^e
Y_e = np.zeros((n_nodes, n_nodes))

for i, row_sub in substations_df.iterrows():
    sub = row_sub["name"]

    # Get index in look up table
    index = sub_look_up.get(sub, None)

    if index is None or sub == "Substation 1":
      continue

    Rg = row_sub.grounding_resistance
    Y_rg = 1 / (3 * Rg) # Divide by 3 to get admittance per phase

    Y_e[index, index] = Y_rg

# Reduced Y_e matrix
Y_e = Y_e[np.ix_(non_zero_indices, non_zero_indices)]

pd.DataFrame(Y_e)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**$Y^e$ from LPm-paper**


## Total admittances of the network
This is obtained by summing the earthing and network admittances.

In [35]:
# Step 1: Add Y^e and Y^n
Y_total = Y_e + Y_n

# Step 2: Invert the result
try:
    Y_inv = np.linalg.pinv(Y_total)
except np.linalg.LinAlgError:
    print("Matrix is singular and cannot be inverted.")
    exit()
pd.DataFrame(Y_inv)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2.060459,0.280527,0.081773,0.080040,0.321828,0.316596,0.070371,0.069627,0.035579,0.023103,0.003024,2.060459,0.258948,0.076229,0.319768,0.069380,0.028463,0.002592
1,0.280527,0.501670,0.047274,0.047616,0.060199,0.059596,0.028391,0.028829,0.013188,0.008564,0.001121,0.280527,0.463080,0.045349,0.059912,0.028283,0.010550,0.000961
2,0.081773,0.047274,0.356748,0.343110,0.116144,0.116627,0.043784,0.043989,0.068468,0.044459,0.005819,0.081773,0.043638,0.326771,0.116018,0.043432,0.054774,0.004988
3,0.080040,0.047616,0.343110,0.349220,0.112344,0.112807,0.043253,0.043489,0.065976,0.042841,0.005607,0.080040,0.043954,0.332591,0.112221,0.042918,0.052781,0.004806
4,0.321828,0.060199,0.116144,0.112344,0.582491,0.572648,0.112197,0.110274,0.057888,0.037589,0.004920,0.321828,0.055569,0.106994,0.578665,0.110325,0.046310,0.004217
5,0.316596,0.059596,0.116627,0.112807,0.572648,0.575336,0.112694,0.110761,0.058147,0.037757,0.004942,0.316596,0.055012,0.107436,0.572105,0.110813,0.046517,0.004236
6,0.070371,0.028391,0.043784,0.043253,0.112197,0.112694,0.231466,0.226754,0.031830,0.020669,0.002705,0.070371,0.026207,0.041194,0.112083,0.227308,0.025464,0.002319
7,0.069627,0.028829,0.043989,0.043489,0.110274,0.110761,0.226754,0.234501,0.031382,0.020378,0.002667,0.069627,0.026611,0.041418,0.110162,0.227577,0.025106,0.002286
8,0.035579,0.013188,0.068468,0.065976,0.057888,0.058147,0.031830,0.031382,0.258206,0.167665,0.021946,0.035579,0.012174,0.062834,0.057829,0.031338,0.206565,0.018811
9,0.023103,0.008564,0.044459,0.042841,0.037589,0.037757,0.020669,0.020378,0.167665,1.046525,0.136980,0.023103,0.007905,0.040801,0.037551,0.020349,0.134132,0.117411


**$Y_{total}^{-1}\ or \  (Y^e + Y^n)^{-1}$ from LPm-paper**

# Current Injections

The electrostatic potentials computed using a uniform field of 1 V/km are the voltage sources that drive the GIC. We can compute the equivalent current sources of the lines using Ohm's law.

$$j_{nk} = V_{nk}/R_{nk}$$

where $j_{nk}$ represents the current sources of the transmission line between nodes $n$ and $k$. $V_{nk}$ and $R_{nk}$ are the transmission line voltage sources and resistances, respectively.

The current injections into node $k$, denoted as $j_k$, are obtained using Kirchhoff's Current Law (KCL), which states that the sum of currents entering a node equals the sum of currents leaving the node:

$$j_k = \sum_{i=1}^N j_{nk} = -\sum_{i=1}^N j_{kn}$$

We do not derive the formula to compute the nodal currents as it is explicitly shown in [Boteler-LPm work](https://angeo.copernicus.org/articles/40/205/2022/angeo-40-205-2022.html). From this work, we skip equations 6 to 16 of the LP model and utilize equation 22 to solve for nodal voltages:

$$V^n = (Y^e + Y^n)^{-1}J^e$$

where $J^e$ refers to total nodal current sources in the system, and $V^n$ represents the solved nodal voltages.

In our model, $Y_{total}$ is the sum of earthing ($Y^e$) and network admittances ($Y^n$). In the Boteler-LPm model, we use the derived nodal voltages and earthing impedances to solve for currents into the nodes ($I^e$):

$$I^e = (Z^e)^{-1}V^n$$

To compute the GIC flowing in the windings and transmission lines, we use:

$$i_{kn} = y_{kn}(v_k-v_n)$$

where $k$ and $n$ represent the buses or ground nodes in the substations.

In [32]:
def calculate_potential(distance:float, E:float):
  # The potential -E.dl evaluated from from bus to to bus
  V = E * distance
  return V

if NERC:
  e_field = 10
else:
  e_field = 1

df_lines["Vx"] = df_lines["Ex"].apply(lambda x: calculate_potential(x, e_field))
df_lines["Vy"] = df_lines["Ey"].apply(lambda x: calculate_potential(x, e_field))

# Net V
# Net V is EnLn + EELE
df_lines['V_net'] = df_lines["Ex"] * e_field + df_lines["Ey"] * e_field

# Eastward and Northward current sources
df_lines["Jx"] = (phases*df_lines['Vx']) / df_lines["R"]
df_lines["Jy"] = (phases*df_lines['Vy']) / df_lines["R"]
df_lines["net_j"] = (phases*df_lines['V_net']) / df_lines["R"]

# Solving for current injections
# Number of buses
n_buses = n_nodes

# Using KCL to solve for current sources
# Initialize injection currents for x and y components
injection_currents_x = np.zeros(n_buses)
injection_currents_y = np.zeros(n_buses)
injections_net = np.zeros(n_buses)

# Calculate line currents and injection currents
for I, row in df_lines.iterrows():
    # Calculate the currents in the line for x and y components
    I_line_x = row["Jx"]
    I_line_y = row["Jy"]
    I_net = row["net_j"]

    # Get i and j
    i = sub_look_up.get(row['from_bus'])
    j = sub_look_up.get(row['to_bus'])


    # Add the currents to the injection currents
    injection_currents_x[i] -= I_line_x
    injection_currents_x[j] += I_line_x

    injection_currents_y[i] -= I_line_y
    injection_currents_y[j] += I_line_y

    injections_net[i] -= I_net
    injections_net[j] += I_net

# Eliminate zero indices
injection_currents_x = injection_currents_x[non_zero_indices]
injection_currents_y = injection_currents_y[non_zero_indices]
injections_net = injections_net[non_zero_indices]

# Print the injection currents
print("Injection Currents (A):")
for i in range(len(injection_currents_x)):
    print(f"Bus {i+1}: X: {injection_currents_x[i]:.2f}, Y: {injection_currents_y[i]:.2f}, Net: {injections_net[i]:.2f}")

Injection Currents (A):
Bus 1: X: -60.73, Y: -19.91, Net: -80.64
Bus 2: X: -29.26, Y: 56.17, Net: 26.92
Bus 3: X: -65.20, Y: 66.74, Net: 1.54
Bus 4: X: 32.84, Y: 25.86, Net: 58.70
Bus 5: X: 34.25, Y: -2.08, Net: 32.18
Bus 6: X: -244.97, Y: 61.19, Net: -183.78
Bus 7: X: 47.23, Y: -105.05, Net: -57.82
Bus 8: X: 22.89, Y: -60.05, Net: -37.15
Bus 9: X: 276.92, Y: -90.26, Net: 186.67
Bus 10: X: -82.73, Y: 70.08, Net: -12.65
Bus 11: X: 68.75, Y: -2.71, Net: 66.04
Bus 12: X: 0.00, Y: 0.00, Net: 0.00
Bus 13: X: 0.00, Y: 0.00, Net: 0.00
Bus 14: X: 0.00, Y: 0.00, Net: 0.00
Bus 15: X: 0.00, Y: 0.00, Net: 0.00
Bus 16: X: 0.00, Y: 0.00, Net: 0.00
Bus 17: X: 0.00, Y: 0.00, Net: 0.00
Bus 18: X: 0.00, Y: 0.00, Net: 0.00


**$J^e$ from LPm-paper**

## Solve for Nodal Voltages with Sparse Matrix Techniques

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.linalg import cholesky, solve
from scipy.sparse import eye
import time

t0 = time.time()
# Solve for nodal voltages
def solve_eqn(Y, injection_currents):
    # Solve for nodal voltages
    # Due to size of the matrix add regularization term
    # Skews the results slightly but is fairly faster
    Y_reg = Y.T @ Y + 1e-20 * eye(Y.shape[1])
    V_nodal = spsolve(Y_reg, Y.T @ injection_currents)

    # Return nodal voltages
    return V_nodal

Vx_ = solve_eqn(Y_total, injection_currents_x)
Vy_ = solve_eqn(Y_total, injection_currents_y)

print("Nodal voltages V_nx:", Vx_)
print("Nodal voltages V_ny:", Vy_)

print(time.time() - t0)

Nodal voltages V_nx: [-189.5400166   -40.73235615  -24.22122716  -22.83439071 -124.5100128
 -125.37515707   -7.38860027   -6.25254646   44.19087042  -40.44242313
   15.61846189 -189.5400166   -37.59909799  -21.74703877 -123.99401445
   -6.66747957   35.35269634   13.38725305]
Nodal voltages V_ny: [-12.41847747  25.1156385   30.17095799  29.4455498   20.09740228
  20.38771094 -29.08869264 -29.11205897  -7.18012633  60.7156251
   7.12329742 -12.41847747  23.18366631  28.04338076  20.0377785
 -27.97888382  -5.74410106   6.1056835 ]
0.01769399642944336


<ipython-input-16-6e89187f3769>:13: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  V_nodal = spsolve(Y_reg, Y.T @ injection_currents)


**$V^n$ from LPm-paper**

## Spsolve with Cholesky
Since we have a large sparse matrix for the grid of the entire US, we will adopt this method to solve for currents and voltages into the nodes.
For a positive-definite symmetric matrix the [L] matrix can be chosen such that the $U$ is $L^T$. In this case, we can write $Y_{total}$ using the Cholesky decomposition:

$$
LL^T = Y_{total}.
$$

This Cholesky decomposition to solve the linear set is:

$$
(Y_{total})V^n = (LL^T)V^n = L(L^TV^n) = J^e,
$$

Then solving for the column matrix [P] is:

$$
LP = J^e
$$

and then solving:

$$
L^TV^n = P
$$

Then use $V^n$ to solve for GIC into the nodes using cholesky with earthing impedances

In [44]:
# Using cholesky decomposition
t0 = time.time()

def get_and_solve_cholesky(Y, Je):

  regularization=1e-6
  Y_reg = Y + np.eye(Y.shape[0]) * regularization
  # Cholesky decomposition of Y_n
  L = cholesky(Y_reg, lower=True)

  # Step 1: Solve L * P = Je
  P = solve(L, Je)

  # Step 2: Solve L^T * V_n = P to get nodal voltages V_n
  V_n = solve(L.T, P)

  return V_n

V_nx = get_and_solve_cholesky(Y_total, injection_currents_x)
V_ny = get_and_solve_cholesky(Y_total, injection_currents_y)

# V net
V_net = get_and_solve_cholesky(Y_total, injections_net)

print("Nodal voltages V_nx:", V_nx)
print("Nodal voltages V_ny:", V_ny)
print("Nodal voltages Net:", V_net)

Nodal voltages V_nx: [-189.50974299  -40.74737007  -24.17704437  -22.79365892 -124.4359498
 -125.01854158   -6.88692306   -6.60047975   44.25728816  -40.39926059
   15.6241032  -189.50972404  -37.61295525  -21.70824597 -124.31767365
   -6.70529279   35.4058284    13.39208788]
Nodal voltages V_ny: [-12.19946023  25.19764641  30.28588508  29.5593312   20.45283788
  20.65915646 -28.46612811 -28.47114674  -7.09662595  60.7697871
   7.13038292 -12.19945901  23.25936485  28.1517432   20.46217699
 -28.18627275  -5.67730042   6.11175653]
Nodal voltages Net: [-201.70920321  -15.54972365    6.10884071    6.76567228 -103.98311192
 -104.35938512  -35.35305117  -35.07162648   37.16066221   20.3705265
   22.75448612 -201.70918304  -14.3535904     6.44349722 -103.85549665
  -34.89156553   29.72852798   19.50384441]


## Calculate Transmission Lines GIC
GIC flowing in the primaries and tranmsmission lines is computed using,
$$i_{kn} = y_{kn}(v_k-v_n)$$

In [42]:
df_lines_copy = df_lines.copy()

# Get the nodal voltages bus ids
df_lines_copy["from_bus"] = df_lines_copy["from_bus"].apply(lambda x: sub_look_up.get(x))
df_lines_copy["to_bus"] = df_lines_copy["to_bus"].apply(lambda x: sub_look_up.get(x))

def calculate_GIC(df, V_nodal, col):
    """
    Calculate the Ground Induced Current (GIC) for a given dataframe.
    Parameters:
    - df (pandas.DataFrame): The input dataframe containing the transmission line data.
    - V_nodal (numpy.ndarray): The nodal voltages.
    - col (str): The column name representing the GIC values.
    Returns:
    - df (pandas.DataFrame): The input dataframe with an additional column representing the calculated GIC values.
    """

    V_all = np.zeros(n_nodes)
    V_all[non_zero_indices] = V_nodal

    bus_n = df["from_bus"].values
    bus_k = df["to_bus"].values

    y_nk = 1 / df["R"].values

    j_nk =  (df[col].values) * y_nk

    # Get the nodal voltages
    vn = V_all[bus_n]
    vk = V_all[bus_k]

    # Solving for transmission lines GIC
    i_nk = np.round(j_nk + (vn - vk) * y_nk, 2)

    df[f"{col}_i_nk"] = i_nk

    return df

df = calculate_GIC(df_lines_copy, V_nx, "Vx")
df = calculate_GIC(df, V_ny, "Vy")
df = calculate_GIC(df, V_net, "V_net")

df = df.round(2)
df[['name', 'from_bus', 'to_bus', 'R', 'V', 'origin_sub', 'to_sub','Vx_i_nk', 'Vy_i_nk', 'V_net_i_nk']]

,name,from_bus,to_bus,R,V,origin_sub,to_sub,Vx_i_nk,Vy_i_nk,V_net_i_nk
0,1,1,7,3.510000e+00,345,Substation 1,Substation 4,15.72,-11.37,4.35
1,2,1,2,3.520000e+00,345,Substation 1,Substation 2,-15.72,11.37,-4.35
2,3,5,8,1.990000e+00,500,Substation 3,Substation 4,-14.02,-17.95,-31.97
3,4,2,6,4.660000e+00,345,Substation 2,Substation 3,29.35,-9.40,19.95
4,5,8,9,2.350000e+00,500,Substation 4,Substation 5,5.39,-19.01,-13.62
5,6,8,9,2.350000e+00,500,Substation 4,Substation 5,5.39,-19.01,-13.62
6,7,9,11,2.980000e+00,500,Substation 5,Substation 6,47.11,17.94,65.05
7,8,9,14,1.000000e+10,500,Substation 5,Substation 7,0.00,0.00,0.00
8,9,11,14,1.440000e+00,500,Substation 6,Substation 7,44.64,20.37,65.01
9,10,8,11,4.670000e+00,500,Substation 4,Substation 6,32.36,1.88,34.24


Vx_i_nk and Vy_i_nk are the GIC for the eastward and northward components of the E field in Amps/phase. The results from this method is compared with the results in the Horton grid 2013.


## Calculate Transformer GICs

In [43]:
df_transformers_copy = df_transformers.copy()

def calc_trafo_gic(sub_look_up, df_transformers, V_nodal, sub_ref):

  gic = {}

  V_all = np.zeros(n_nodes)
  V_all[non_zero_indices] = V_nodal

  # Process transformers and build admittance matrix
  for bus, bus_idx in sub_look_up.items():
    sub = find_substation_name(bus, sub_ref)

    # Filter transformers for current bus
    trafos = df_transformers[(df_transformers["bus1"] == bus)]

    if len(trafos) == 0 or sub == "Substation 7":
        continue

    # Process each transformer
    for _, trafo in trafos.iterrows():
      # Extract parameters
      bus1 = trafo["bus1"]
      bus2 = trafo["bus2"]
      neutral_point = trafo["sub"]  # Neutral point node (for auto-transformers)
      W1 = trafo["W1"]  # Impedance for Winding 1 (Primary, Series)
      W2 = trafo["W2"]  # Impedance for Winding 2 (Secondary, if available)

      trafo_name = trafo["name"]

      trafo_type = trafo["type"]
      bus1_idx = sub_look_up[bus1]
      neutral_idx = sub_look_up[neutral_point] if neutral_point in sub_look_up else None
      bus2_idx = sub_look_up[bus2]

      if trafo_type == "GSU":
        Y_w1 = 1 / W1  # Primary winding admittance
        i_k = (V_all[bus1_idx] - V_all[neutral_idx]) * Y_w1
        gic[trafo_name] = {"HV":i_k}

      elif trafo_type == "Tee":
        # Y_pri = phases / (row['W1'])
        # Y_sec = phases / (row['W2'])
        # add_admittance(Y, bus_n, np_bus, Y_pri)
        # add_admittance(Y, bus_k, np_bus, Y_sec)
        continue

      elif trafo_type == "GSU w/ GIC BD":
        Y_w1 = 1 / W1  # Primary winding admittance
        i_k = (V_all[bus1_idx] - V_all[neutral_idx]) * Y_w1
        gic[trafo_name] = {"HV":i_k}

      elif trafo_type == "Auto":
        Y_series = 1 / W1
        Y_common = 1 / W2
        I_s = (V_all[bus1_idx] - V_all[bus2_idx]) * Y_series
        I_c = (V_all[bus2_idx] -  V_all[neutral_idx]) * Y_common
        gic[trafo_name] = {
            "Series": I_s,
            "Common": I_c
        }

      elif trafo_type in ["GY-GY-D", "GY-GY"]:
        Y_primary = 1 / W1
        Y_secondary = 1 / W2
        add_admittance(Y, bus1_idx, neutral_idx, Y_primary)
        add_admittance(Y, bus2_idx, neutral_idx, Y_secondary)
        I_w1 = (V_all[bus1_idx] - V_all[neutral_idx]) * Y_primary
        I_w2 = (V_all[bus2_idx] - V_all[neutral_idx]) * Y_secondary
        gic[trafo_name] = {
            "HV": I_w1,
            "LV": I_w2
        }

  return gic

gic_dict_x = calc_trafo_gic(sub_look_up, df_transformers_copy, V_nx, sub_ref)
gic_dict_y = calc_trafo_gic(sub_look_up, df_transformers_copy, V_ny, sub_ref)
gic_dict_net = calc_trafo_gic(sub_look_up, df_transformers_copy, V_net, sub_ref)

# Flatten the Northward GIC dictionary into a list of tuples (Transformer, Winding, GIC_vals)
data_x = [(trafo, winding, gic) for trafo, windings in gic_dict_x.items() for winding, gic in windings.items()]

# Create DataFrame for Northward (X) electric field
df_winding_gic_x = pd.DataFrame(data_x, columns=['Transformer', 'Winding', 'E_field_East'])

# Flatten the Eastward GIC dictionary into a list of tuples (Transformer, Winding, GIC_vals)
data_y = [(trafo, winding, gic) for trafo, windings in gic_dict_y.items() for winding, gic in windings.items()]

# Create DataFrame for Eastward (Y) electric field
df_winding_gic_y = pd.DataFrame(data_y, columns=['Transformer', 'Winding', 'E_field_North'])

# Flatten the net GIC dictionary into a list of tuples (Transformer, Winding, GIC_vals)
data_net = [(trafo, winding, gic) for trafo, windings in gic_dict_net.items() for winding, gic in windings.items()]

# Create DataFrame for net electric field
df_winding_gic_net = pd.DataFrame(data_net, columns=['Transformer', 'Winding', 'E_field_Net'])

# Merge the two DataFrames on Transformer and Winding columns
df_combined = pd.merge(df_winding_gic_y, df_winding_gic_x, on=['Transformer', 'Winding'], how='outer')

df_combined = pd.merge(df_combined, df_winding_gic_net, on=['Transformer', 'Winding'], how='outer')

# Sorting index
df_combined["index"] = df_combined.Transformer.apply(lambda x: eval(x[1:]))

df_combined = df_combined.sort_values(by=["index", "Winding"]).reset_index(drop=True)

# Display the combined DataFrame
df_combined.round(2)

,Transformer,Winding,E_field_North,E_field_East,E_field_Net,index
0,T1,HV,-0.00,-0.00,-0.00,1
1,T2,HV,0.98,-3.50,-2.52,2
2,T2,LV,-0.09,-1.18,-1.28,2
3,T3,HV,19.38,-31.34,-11.96,3
4,T4,HV,19.38,-31.34,-11.96,4
5,T5,Common,23.46,-18.09,5.37,5
6,T5,Series,18.16,-34.58,-16.42,5
7,T6,HV,-9.46,59.01,49.55,6
8,T7,HV,-9.46,59.01,49.55,7
9,T8,HV,-7.00,-4.54,-11.54,8


## Solve for Grounding GICs

In [57]:
def solve_total_nodal_gic(Y_e, Vn):

  # Current flowing to the ground
  with np.errstate(divide='ignore', invalid='ignore'):
      Z_e = 1 / Y_e
      Z_e[np.isinf(Z_e)] = 0  # Replace inf with 0

  i_g = solve_eqn(Z_e, Vn) # Per phase
  return i_g*3 # Total

# Due to Eastward Components
igx = solve_total_nodal_gic(Y_e, V_nx)

# Due to Northward Geoelectric field
igy = solve_total_nodal_gic(Y_e, V_ny)

pd.DataFrame(
    (np.vstack([igx, igy])).T,
    columns=["GIC\nEastward field", "GIC\nNorthward field"]
)

<ipython-input-16-6e89187f3769>:13: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  V_nodal = spsolve(Y_reg, Y.T @ injection_currents)


,GIC\nEastward field,GIC\nNorthward field
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,0.000000
9,0.000000,0.000000


**Comparison with Horton 2013 results**
